In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn import preprocessing

In [4]:
data = pd.read_excel('sample dataset.xlsx')
data_copy = data.copy()
data_copy.head()

,id2,universi,region,cluster,cross,sex,birth_ye,sample_c,h_18,h_19,...,n_84,n_85,n_86,n_87,n_88,n_89,n_90,n_91,n_92,n_93
0,1101118501,tabriz,urban,1.0,Elemantry,Boy,85.0,1.0,9.0,11.0,...,never,never,never,never,never,never,no,never,good,10.0
1,1101118502,tabriz,urban,1.0,Elemantry,Boy,85.0,2.0,NaN,NaN,...,never,never,every month,never,never,never,be khater nadaram,never,good,10.0
2,1101118505,tabriz,urban,1.0,Elemantry,Boy,85.0,5.0,7.3,9.3,...,never,never,never,never,never,never,be khater nadaram,Sometimes,perfect,NaN
3,1101118507,tabriz,urban,1.0,Elemantry,Boy,85.0,7.0,9.0,9.0,...,never,never,never,never,never,never,no,never,good,10.0
4,1101118508,tabriz,urban,1.0,Elemantry,Boy,85.0,8.0,8.3,9.0,...,never,never,never,NaN,never,NaN,yes,Sometimes,Average,NaN


In [3]:
# Identify columns with the same value in all rows
columns_to_drop = [col for col in data_copy.columns if data_copy[col].nunique() == 1]
# Drop these columns
data_cleaned = data_copy.drop(columns= columns_to_drop)

#print(data_cleaned.shape)
print(data_cleaned.head())

          id2  cluster      cross  sex  birth_ye  sample_c  h_18  h_19   h_20   
0  1101118501      1.0  Elemantry  Boy      85.0       1.0   9.0  11.0  6 roz  \
1  1101118502      1.0  Elemantry  Boy      85.0       2.0   NaN   NaN    NaN   
2  1101118505      1.0  Elemantry  Boy      85.0       5.0   7.3   9.3  6 roz   
3  1101118507      1.0  Elemantry  Boy      85.0       7.0   9.0   9.0  6 roz   
4  1101118508      1.0  Elemantry  Boy      85.0       8.0   8.3   9.0  5 roz   

         h_21  ...   n_84   n_85         n_86   n_87   n_88   n_89   
0  nemikhoram  ...  never  never        never  never  never  never  \
1         NaN  ...  never  never  every month  never  never  never   
2    mikhoram  ...  never  never        never  never  never  never   
3    mikhoram  ...  never  never        never  never  never  never   
4    mikhoram  ...  never  never        never    NaN  never    NaN   

                n_90       n_91     n_92  n_93  
0                 no      never     good  1

In [4]:
M = data_copy.shape[0]
N = data_copy.shape[1]


In [5]:
def convert_float_to_time(time):
    new_time = time
    if pd.isnull(time) == False:
        hour = math.floor(time)
        minute = round((time - hour) * 100)
        new_time = f"{hour:02d}:{minute:02d}"
    return new_time


In [6]:
def repair_time(question_num):
    for i in range(M):
        new_time1 = convert_float_to_time(data_copy[question_num][i])
        data_copy.at[i , question_num] = new_time1
    data_cleaned[question_num] = pd.to_datetime(data_copy[question_num] , format = '%H:%M')


In [7]:
def average_of_time(question_num):
    avg = data_cleaned[question_num].mean()
    hour = avg.hour
    minute = avg.minute
    if minute > 45:
        minute = 0
        hour = hour +1
    elif minute < 15:
        minute = 0
    else:
        minute = 30

    avg = pd.to_datetime(f"{hour:02d}:{minute:02d}" , format = '%H:%M')
    return avg

In [8]:
repair_time('h_18')

In [9]:
repair_time('h_19')

In [10]:
avg18 = average_of_time('h_18')

In [11]:
avg19 = average_of_time('h_19')

In [12]:
avg19 - avg18

Timedelta('0 days 01:30:00')

In [677]:
for i in range(M):
    if pd.isna(data_cleaned['h_18'][i]) and pd.isna(data_cleaned['h_19'][i]):
        data_cleaned['h_18'][i] = avg18
        data_cleaned['h_19'][i] = avg19
    elif pd.isna(data_cleaned['h_18'][i]):
        data_cleaned['h_18'][i] = (data_cleaned['h_19'][i] - (avg19 - avg18))
    elif pd.isna(data_cleaned['h_19'][i]):
        data_cleaned['h_19'][i] = (data_cleaned['h_18'][i] + (avg19 - avg18))

C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\616965990.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['h_18'][i] = avg18
C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\616965990.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['h_19'][i] = avg19
C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\616965990.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['h_18'][i] = (data_cleaned['h

In [678]:
data_cleaned['h_18'] = data_cleaned['h_18'].dt.time
data_cleaned['h_19'] = data_cleaned['h_19'].dt.time

In [679]:
def repair_suffix_number(question_num , suffix , substitute):
    data_cleaned[question_num] = data_copy[question_num].str.replace(substitute , f'{0} {suffix}')
    for i in range(len(data_cleaned[question_num])):
        if pd.isna(data_cleaned[question_num][i]) == False:
            data_cleaned[question_num][i] =int((data_cleaned[question_num][i])[0])

In [680]:
repair_suffix_number('h_20' , 'roz' ,'aslan nemikhoram')

C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\4122256761.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[question_num][i] =int((data_cleaned[question_num][i])[0])
C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\4122256761.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[question_num][i] =int((data_cleaned[question_num][i])[0])
C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\4122256761.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [681]:
most_value = data_cleaned['h_20'].mode()[0]
avg_day_school =math.floor(data_cleaned['h_20'].mean())
data_cleaned['h_20'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_20
Non-Null Count  Dtype 
--------------  ----- 
192 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [682]:
elementry_row = data_cleaned[data_cleaned['cross'] == 'Elemantry']
avg_elementry = elementry_row['h_20'].mean()


In [683]:
intermidiate_row = data_cleaned[data_cleaned['cross'] == 'intermidiate']
avg_intermidiate = intermidiate_row['h_20'].mean()

In [684]:
print(avg_intermidiate , avg_elementry)

5.135135135135135 4.686440677966102


In [685]:
data_cleaned['h_20'] = data_cleaned['h_20'].fillna(avg_day_school)

In [686]:
data_cleaned.loc[(data_cleaned['h_20'] <= 2) & (data_cleaned['h_21'].isna()),'h_21'] = 'nemikhoram'


In [687]:
data_cleaned['h_21'] = data_cleaned['h_21'].fillna('mikhoram')

In [688]:
data_cleaned['h_21'] = data_cleaned['h_21'].replace({'nemikhoram' : False , 'mikhoram' : True})

In [689]:
repair_time('h_22')

In [690]:
avg22 = average_of_time('h_22')

In [691]:
data_cleaned['h_22'] =  data_cleaned['h_22'].fillna(pd.to_datetime(f"{avg22.hour:02d}:{0:02d}" , format="%H:%M"))
data_cleaned['h_22'] = data_cleaned['h_22'].dt.time

In [692]:
repair_time('h_23')

In [693]:
avg23 = average_of_time('h_23')

In [694]:
data_cleaned['h_23'] =  data_cleaned['h_23'].fillna(pd.to_datetime(f"{avg23.hour:02d}:{0:02d}" , format="%H:%M"))
data_cleaned['h_23'] = data_cleaned['h_23'].dt.time

In [695]:
repair_suffix_number('h_24' , 'roz' , 'aslan nemikhoram')

C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\4122256761.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[question_num][i] =int((data_cleaned[question_num][i])[0])
C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\4122256761.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[question_num][i] =int((data_cleaned[question_num][i])[0])
C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\4122256761.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [696]:
data_cleaned['h_24'] = data_cleaned['h_24'].fillna(int(data_cleaned['h_24'].mean()))

In [697]:
data_cleaned

,id2,cluster,cross,sex,birth_ye,sample_c,h_18,h_19,h_20,h_21,...,n_84,n_85,n_86,n_87,n_88,n_89,n_90,n_91,n_92,n_93
0,1101118501,1.0,Elemantry,Boy,85.0,1.0,09:00:00,11:00:00,6,False,...,never,never,never,never,never,never,no,never,good,10.0
1,1101118502,1.0,Elemantry,Boy,85.0,2.0,08:00:00,09:30:00,4,True,...,never,never,every month,never,never,never,be khater nadaram,never,good,10.0
2,1101118505,1.0,Elemantry,Boy,85.0,5.0,07:30:00,09:30:00,6,True,...,never,never,never,never,never,never,be khater nadaram,Sometimes,perfect,NaN
3,1101118507,1.0,Elemantry,Boy,85.0,7.0,09:00:00,09:00:00,6,True,...,never,never,never,never,never,never,no,never,good,10.0
4,1101118508,1.0,Elemantry,Boy,85.0,8.0,08:30:00,09:00:00,5,True,...,never,never,never,NaN,never,NaN,yes,Sometimes,Average,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1123128606,23.0,Elemantry,Girl,86.0,6.0,08:00:00,10:00:00,1,True,...,never,everyweek,everyweek,everyweek,never,never,no,never,perfect,10.0
196,1123128607,23.0,Elemantry,Girl,86.0,7.0,08:00:00,10:00:00,6,True,...,never,everyday,never,everyweek,never,never,no,never,perfect,10.0
197,1123128608,23.0,Elemantry,Girl,86.0,8.0,08:00:00,10:00:00,6,True,...,never,never,everyweek,never,never,never,no,never,perfect,8.0
198,1123128701,23.0,Elemantry,Girl,87.0,1.0,08:00:00,10:00:00,1,True,...,never,never,More than once a week,never,never,never,no,never,perfect,10.0


In [698]:
data_cleaned.loc[(data_cleaned['h_24'] < 2) & (data_cleaned['h_25'].isna()),'h_25'] = 'nemikhoram'
data_cleaned['h_25'] = data_cleaned['h_25'].fillna('mikhoram')
data_cleaned['h_25'] = data_cleaned['h_25'].replace({'nemikhoram' : False , 'mikhoram' : True})

In [699]:
data_copy['h_26'] = data_copy['h_26'].replace(24 , 0)

In [700]:
repair_time('h_26')

In [701]:
avg26 = average_of_time('h_26')

In [702]:
data_cleaned['h_26'] = data_cleaned['h_26'].fillna(avg26)

In [703]:
data_cleaned['h_26'] = data_cleaned['h_26'].dt.time

In [704]:
data_copy['h_27'] = data_copy['h_27'].replace(24 , 0)
repair_time('h_27')
avg26 = average_of_time('h_27')
data_cleaned['h_27'] = data_cleaned['h_27'].fillna(avg26)
data_cleaned['h_27'] = data_cleaned['h_27'].dt.time

In [705]:
repair_suffix_number('h_28' , 'shab' , 'aslan nemikhoram')

C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\4122256761.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[question_num][i] =int((data_cleaned[question_num][i])[0])
C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\4122256761.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[question_num][i] =int((data_cleaned[question_num][i])[0])
C:\Users\Asus\AppData\Local\Temp\ipykernel_22088\4122256761.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [706]:
data_cleaned['h_28']

0        6
1      NaN
2        6
3        6
4        6
      ... 
195      6
196      6
197      6
198      6
199      6
Name: h_28, Length: 200, dtype: object

In [707]:
data_cleaned['h_28'] = data_cleaned['h_28'].fillna(int(data_cleaned['h_28'].mean()))

In [708]:
data_cleaned['h_28'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_28
Non-Null Count  Dtype
--------------  -----
200 non-null    int64
dtypes: int64(1)
memory usage: 1.7 KB


In [709]:
data_cleaned['h_29'] = data_cleaned['h_29'].fillna("mikhoram")

In [710]:
data_cleaned['h_29']

0      mikhoram
1      mikhoram
2      mikhoram
3      mikhoram
4      mikhoram
         ...   
195    mikhoram
196    mikhoram
197    mikhoram
198    mikhoram
199    mikhoram
Name: h_29, Length: 200, dtype: object

In [711]:
def fill_nan(question_num):
    value =  data_cleaned[question_num].mode()[0]
    print(type(value))
    data_cleaned[question_num].fillna(value , inplace=True)
    data_cleaned[question_num].info()

In [712]:
def ordinary_encoding(question_num):
    fill_nan(question_num)
    #data_cleaned[question_num] = data_cleaned[question_num].apply(str)
    categories = [['never', 'rarely', 'Daily', 'weekly']]
    encoder = preprocessing.OrdinalEncoder(categories= categories)
    data = data_cleaned[question_num].values.reshape(-1 , 1)
    data_cleaned[question_num] =  encoder.fit_transform(data)


In [713]:
ordinary_encoding('h_30')
ordinary_encoding('h_31')
ordinary_encoding('h_32')
ordinary_encoding('h_33')
ordinary_encoding('h_34')

<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_30
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_31
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_32
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_33
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_34
Non-Null Count  Dtype 
--------------  ----- 
200 non-

In [714]:
data_cleaned['h_35_a'] = data_cleaned['h_35_a'].fillna(data_cleaned['h_35_b'])

In [715]:
ordinary_encoding('h_35_a')

<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_35_a
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [716]:
data_cleaned = data_cleaned.drop(columns='h_35_b')

In [717]:
ordinary_encoding('h_36')
ordinary_encoding('h_37')
ordinary_encoding('h_38')
ordinary_encoding('h_39')
ordinary_encoding('h_40')
ordinary_encoding('h_41')
ordinary_encoding('h_42')

<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_36
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_37
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_38
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_39
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_40
Non-Null Count  Dtype 
--------------  ----- 
200 non-

In [718]:
ordinary_encoding('h_43_1')

<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_43_1
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [719]:
ordinary_encoding('h_44_1')
ordinary_encoding('h_45')

<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_44_1
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_45
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [720]:
import re
def extract_number_from_string_to_arr(question_number):
    fill_nan(question_number)
    for i in range(M):
        data_cleaned[question_number][i] = [int(s) for s in re.findall(r'\d+' , data_cleaned[question_number][i])]

In [721]:
extract_number_from_string_to_arr('h_43_2')

<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_43_2
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


,id2,cluster,cross,sex,birth_ye,sample_c,h_18,h_19,h_20,h_21,...,n_84,n_85,n_86,n_87,n_88,n_89,n_90,n_91,n_92,n_93
0,1101118501,1.0,Elemantry,Boy,85.0,1.0,09:00:00,11:00:00,6,False,...,never,never,never,never,never,never,no,never,good,10.0
1,1101118502,1.0,Elemantry,Boy,85.0,2.0,08:00:00,09:30:00,4,True,...,never,never,every month,never,never,never,be khater nadaram,never,good,10.0
2,1101118505,1.0,Elemantry,Boy,85.0,5.0,07:30:00,09:30:00,6,True,...,never,never,never,never,never,never,be khater nadaram,Sometimes,perfect,NaN
3,1101118507,1.0,Elemantry,Boy,85.0,7.0,09:00:00,09:00:00,6,True,...,never,never,never,never,never,never,no,never,good,10.0
4,1101118508,1.0,Elemantry,Boy,85.0,8.0,08:30:00,09:00:00,5,True,...,never,never,never,NaN,never,NaN,yes,Sometimes,Average,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1123128606,23.0,Elemantry,Girl,86.0,6.0,08:00:00,10:00:00,1,True,...,never,everyweek,everyweek,everyweek,never,never,no,never,perfect,10.0
196,1123128607,23.0,Elemantry,Girl,86.0,7.0,08:00:00,10:00:00,6,True,...,never,everyday,never,everyweek,never,never,no,never,perfect,10.0
197,1123128608,23.0,Elemantry,Girl,86.0,8.0,08:00:00,10:00:00,6,True,...,never,never,everyweek,never,never,never,no,never,perfect,8.0
198,1123128701,23.0,Elemantry,Girl,87.0,1.0,08:00:00,10:00:00,1,True,...,never,never,More than once a week,never,never,never,no,never,perfect,10.0


In [722]:
extract_number_from_string_to_arr('h_44_2')
extract_number_from_string_to_arr('h_44_3')

<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_44_2
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_44_3
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [723]:
extract_number_from_string_to_arr('z_52')

<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: z_52
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [724]:
extract_number_from_string_to_arr('z_54')
extract_number_from_string_to_arr('h_57')
extract_number_from_string_to_arr('h_58')
extract_number_from_string_to_arr('h_61')
extract_number_from_string_to_arr('h_62')


<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: z_54
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_57
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_58
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_61
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_62
Non-Null Count  Dtype 
--------------  ----- 
200 non-

In [725]:
extract_number_from_string_to_arr('h_59')

<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_59
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [726]:
extract_number_from_string_to_arr('h_60')

<class 'str'>
<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: h_60
Non-Null Count  Dtype 
--------------  ----- 
200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [729]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 72 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id2       200 non-null    int64  
 1   cluster   199 non-null    float64
 2   cross     199 non-null    object 
 3   sex       199 non-null    object 
 4   birth_ye  199 non-null    float64
 5   sample_c  199 non-null    float64
 6   h_18      200 non-null    object 
 7   h_19      200 non-null    object 
 8   h_20      200 non-null    int64  
 9   h_21      200 non-null    bool   
 10  h_22      200 non-null    object 
 11  h_23      200 non-null    object 
 12  h_24      200 non-null    int64  
 13  h_25      200 non-null    bool   
 14  h_26      200 non-null    object 
 15  h_27      200 non-null    object 
 16  h_28      200 non-null    int64  
 17  h_29      200 non-null    object 
 18  h_30      200 non-null    float64
 19  h_31      200 non-null    float64
 20  h_32      200 non-null    float6